# Giriş işlemleri

In [1]:
import os
is_colab = False # eğer colab kullanıyorsanız True yapın
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
root_path = os.getcwd()  # dosyanın bulunduğu klasör
print(root_path)

c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu


## Gerekli kütüphaneleri yükle

In [2]:
import subprocess

# gereksinimler dosyasının yolu
requirements_path = os.path.join(root_path, "gereksinimler.txt")

# gereksinimler yükleniyor
try:
    print("Gereksinimler yükleniyor...")
    subprocess.run(["pip", "install", "-r", requirements_path], check=True)
    print("Gereksinimler başarıyla yüklendi.")
except subprocess.CalledProcessError:
    print("Gereksinimler yüklenirken hata oluştu.")
except FileNotFoundError:
    print(f"Dosya bulunamadı: {requirements_path}")

Gereksinimler yükleniyor...
Gereksinimler başarıyla yüklendi.


## Dosya dizinini sistem yoluna ekle

In [3]:
import sys

sys.path.append(root_path)
print(f"'{root_path}' kütüphane arama yoluna eklendi.")

'c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu' kütüphane arama yoluna eklendi.


## Yerel kod dosyalarını içe aktar

In [ ]:
from dosya_islemleri import (save_smilarity_json, load_model, load_dataset, save_top1_top5_results_json,
                            get_all_top1_top5_results, read_embedding_from_file)
from benzerlik_islemleri import generate_similarity_json
from basari_hesapla import evaluate_similarity_results_top1_top5
from gorsellestir import visualize_top1_top5_scores, save_tsne_png
from gomme_islemleri import calculate_and_save_raw_embeddings_from_dataset, tsne_sonuc_olustur

C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cihaz tipini al

In [5]:
import torch
# cihaz tipini al
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz tipi: {device}")

Kullanılan cihaz tipi: cpu


## Modelleri tanımla

In [6]:
model_names = [
    "intfloat/multilingual-e5-base",                                # 278M  - 81
    "ibm-granite/granite-embedding-107m-multilingual",              # 107M  - 48
    "intfloat/multilingual-e5-small",                               # 118M  - 36
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 118M  - 62
    "shibing624/text2vec-base-multilingual",                        # 118M  - 77 
    "ytu-ce-cosmos/turkish-colbert"
    ]

In [7]:
# veri kümesi yükleniyor
df = load_dataset()

Veri kümesi yükleniyor: c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu\..\gsm8k_tr_1000_soru_cevap.csv


# Gömme (embedding) hesaplama işleri

In [8]:
def get_save_prefix(model_name):
    return model_name.replace("/", "_").replace("-", "_")

In [9]:
# her model için
for i, model_name in enumerate(model_names):
        save_prefix = get_save_prefix(model_name)
        model, tokenizer = load_model(model_name, device_type=device)
        calculate_and_save_raw_embeddings_from_dataset(model, tokenizer, df, save_prefix, device)
        print(f"Embeddingler hesaplandı ve kaydedildi: {model_name} ({i+1}/{len(model_names)})")

Model yükleniyor: intfloat/multilingual-e5-base
Model yüklendi. intfloat/multilingual-e5-base
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: intfloat/multilingual-e5-base (1/6)
Model yükleniyor: ibm-granite/granite-embedding-107m-multilingual
Model yüklendi. ibm-granite/granite-embedding-107m-multilingual
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: ibm-granite/granite-embedding-107m-multilingual (2/6)
Model yükleniyor: intfloat/multilingual-e5-small
Model yüklendi. intfloat/multilingual-e5-small
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: intfloat/multilingual-e5-small (3/6)
Model yükleniyor: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Model yüklendi. sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Hali hazırda hesaplanmış gömme vektörleri say

In [10]:
for i, model_name in enumerate(model_names):
    save_prefix = get_save_prefix(model_name)
    raw_embeddings = read_embedding_from_file(save_prefix)
    tsne_res = tsne_sonuc_olustur(raw_embeddings, save_prefix)

intfloat_multilingual_e5_base_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Sistem belirtilen dosyayı bulamıyor
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info 

ibm_granite_granite_embedding_107m_multilingual_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


intfloat_multilingual_e5_small_embeddings.json dosyasından 10 gömme vektörü yüklendi.


...

C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


shibing624_text2vec_base_multilingual_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


ytu_ce_cosmos_turkish_colbert_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
